# Arquitectura de MODFLOW 6.

<p xmlns:cc="http://creativecommons.org/ns#" xmlns:dct="http://purl.org/dc/terms/"><a property="dct:title" rel="cc:attributionURL" href="https://github.com/luiggix/RTWMA/">Arquitectura de MODFLOW 6</a> by <b>Luis M. de la Cruz Salas</b> is licensed under <a href="http://creativecommons.org/licenses/by-sa/4.0/?ref=chooser-v1" target="_blank" rel="license noopener noreferrer" style="display:inline-block;">Attribution-ShareAlike 4.0 International<img style="height:22px!important;margin-left:3px;vertical-align:text-bottom;" src="https://mirrors.creativecommons.org/presskit/icons/cc.svg?ref=chooser-v1"><img style="height:22px!important;margin-left:3px;vertical-align:text-bottom;" src="https://mirrors.creativecommons.org/presskit/icons/by.svg?ref=chooser-v1"><img style="height:22px!important;margin-left:3px;vertical-align:text-bottom;" src="https://mirrors.creativecommons.org/presskit/icons/sa.svg?ref=chooser-v1"></a></p> 

## Componentes.

Una simulación en MODFLOW 6 típicamente se basa en cuatro componentes, véase **[[1](#ref1)]**:
* **Models**. Un modelo resuelve un proceso hidrológico; por ejemplo, el GWF model, resuelve la ecuación de flujo subterráneo usando el método CVFD.
* **Exchange**. Facilita la comunicación entre dos modelos; por ejemplo, un intercambio GWF-GWF, permite que algunas celdas de un modelo GWF estén hidráulicamente conectadas con celdas del otro modelo GWF.
* **Solutions**. Resuelve uno o más modelos hidrológicos, utiliza métodos iterativos para resolver sistemas no-lineales.
* **Timing**. Controla el paso de tiempo y determina el fin de una simulación.


<img src="../figures/components_mf6.png" width=800px>


## Esquema para GWF.


<div>
<img src="../figures/modflow01.png" width=300px hspace="5" vspace="5" style="float: left;"/>

Diagrama esquemático que muestra la configuración del modelo de flujo de agua subterránea (GWF). En la figura se muestra un único modelo GWF que se resuelve mediante una solución numérica, véase **[[2](#ref2)]**.

* El modelo GWF descrito en **[[1](#ref1), [2](#ref2)]** se divide en *paquetes*, como se hacía en versiones anteriores de MODFLOW.
* Un paquete es la parte del modelo que se ocupa de un único aspecto de la simulación.
* Por ejemplo, el paquete *Well* simula el efecto de los pozos y el paquete *River* simula el efecto de los ríos.
* El Modelo GWF contiene muchos paquetes y opciones que el usuario puede o no tener ocasión de utilizar.


### Paquetes

<img src="../figures/gwf_mf6.png" width=500px hspace="5" vspace="5" style="float: left;"/>
<img src="../figures/gwf_mf6_pack.png" width=500px hspace="5" vspace="5" style="float: ;"/>

## Discretización espacial.

El dominio espacial de una simulación en MODFLOW 6 se divide en una rejilla de celdas y provee de  tres paquetes diferentes:

* **DIS**. Paquete para discretización estructurada.
* **DISV**. Paquete para discretización por vértices.
* **DISU**. Paquete para discretización NO estructurada.

Solo se puede usar un paquete de discretización en una simulación con GWF.

La información almacenada en los paquetes de discretización es usada durante la simulación por GWF para:
* calcular la conductividad hidráulica entre celdas conectadas, 
* calcular los volúmenes de celda para los cálculos de almacenamiento,
* convertir los flujos de recarga y evapotranspiración de las dimensiones de L/T a L$^3$/T, por ejemplo

Los paquetes de discretización:
* contienen información acerca de las celdas, como las elevaciones superior (`TOP`) e inferior (`BOTTOM`) de la celda y las dimensiones o áreas de la celda (en vista de planta). 
* definen la conectividad de las celdas, que describe cómo las celdas modelo están conectadas entre sí y las propiedades geométricas de las conexiones.
* En el paquete **DIS**, la conectividad de la celda se describe implícitamente por la estructura de la malla, las dimensiones de la celda en las direcciones de renglones y columnas, y las elevaciones de la parte superior e inferior de la celda.

<img src="../figures/mesh_3D_mf6.png" width=400px hspace="5" vspace="5" style="float: left;"/>
<img src="../figures/mesh_plainview_mf6.png" width=400px hspace="5" vspace="5" style="float: ;"/>


## Discretización temporal.

* La longitud del primer paso de tiempo $\Delta t_1$, está relacionada con `PERLEN`, `NSTP` y `TSMULT` por la siguiente relación:

$$
\Delta t_1= \frac{\text{PERLEN}}{\text{NSTP}} \;\; \text{para} \;\; \text{TSMULT} = 1
$$


y
  
$$
\Delta t_1= \text{PERLEN} \frac{\text{TSMULT} - 1}{\text{TSMULT}^\text{nstp}-1} \;\; \text{para} \;\; \text{TSMULT} \neq 1
$$

* La longitud de cada paso de tiempo sucesivo es calculada multiplicando la longitud del paso de tiempo previo por `TSMULT`.

$$
\Delta t = \Delta t_{old} \text{TSMULT}
$$
  
<img src="../figures/time_step.png" width=500px>

donde 

|Variable|Tipo|Significado|
|--|--|:--|
|`PERLEN` | double| es la longitud del periodo de estrés| 
|`NSTP` |integer| es el número de pasos de tiempo en un periodo de estrés|
|`TSMULT` |double| es el multiplicador para calcular la longitud del paso de tiempo sucesivos|

Véase **[[3](#ref3)]**.

## Referencias

<a name="ref1"></a>
**<font color='Green'>[1] Langevin, C.D., Hughes, J.D., Provost, A.M., Banta, E.R., Niswonger, R.G., and Panday, Sorab, 2017, Documentation for the MODFLOW 6 Groundwater Flow (GWF) Model: U.S. Geological Survey Techniques and Methods, book 6, chap. A55, 197 p., accessed August 4, 2017. </font>** https://doi.org/10.3133/tm6A55.

<a name="ref2"></a>
**<font color='Green'>[2] Langevin, C. D., Hughes, J. D., Provost, A. M., Russcher, M. J., & Panday, S. (2023). MODFLOW as a configurable Multi‐Model Hydrologic Simulator. Ground Water.</font>** https://doi.org/10.1111/gwat.13351

<a name="ref3"></a>
**<font color='Green'>[3] Hughes, J.D., Langevin, C.D., and Banta, E.R., 2017, *Documentation for the MODFLOW 6 framework: U.S. Geological Survey Techniques and Methods*, book 6, chap. A57, 40 p.,</font>** https://doi.org/10.3133/tm6A57. **Timing Module, pp 10,12**.
